# Resnet Single Image Inference Test

- original code : # https://pytorch.org/hub/pytorch_vision_resnet/
- modified by : JPark, Sep. 2022

In [27]:
import torch
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)

model.eval()

# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

# ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt


Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0


--2022-09-13 17:40:06--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.7’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.001s  

2022-09-13 17:40:06 (11.7 MB/s) - ‘imagenet_classes.txt.7’ saved [10472/10472]



In [28]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import time


start = time.time()
for icnt in range(100):
    input_image = Image.open(filename)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model


    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
        
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    #print(output[0])
    
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    #print(probabilities)

    # Read the categories
    with open("imagenet_classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    # Show top categories per image
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        pass
        #print(categories[top5_catid[i]], top5_prob[i].item())
        
end = time.time()
print(end - start)

35.063608169555664


## Test 실험결과

- 100회 반복, M1 MacbookPro14, CPU mode

```csv
model, time[sec]
resnet18, 9.702051162719727 
resnet34, 15.428618907928467
resnet50, 16.397982120513916
resnet101, 25.44774293899536
resnet152, 35.063608169555664
```